In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import numba
import seaborn as sns

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/car_evaluation.csv')
df.head()

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [4]:
df.shape

(1727, 7)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   vhigh    1727 non-null   object
 1   vhigh.1  1727 non-null   object
 2   2        1727 non-null   object
 3   2.1      1727 non-null   object
 4   small    1727 non-null   object
 5   low      1727 non-null   object
 6   unacc    1727 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [6]:
col_name=['buying','maint','doors','persons','lug_boot','safety','class']
df.columns=col_name
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [7]:
df.shape

(1727, 7)

In [8]:
df['buying'].value_counts()

high     432
med      432
low      432
vhigh    431
Name: buying, dtype: int64

In [9]:
for i in df.columns:
  print(i,':\n',df[i].value_counts())

buying :
 high     432
med      432
low      432
vhigh    431
Name: buying, dtype: int64
maint :
 high     432
med      432
low      432
vhigh    431
Name: maint, dtype: int64
doors :
 3        432
4        432
5more    432
2        431
Name: doors, dtype: int64
persons :
 4       576
more    576
2       575
Name: persons, dtype: int64
lug_boot :
 med      576
big      576
small    575
Name: lug_boot, dtype: int64
safety :
 med     576
high    576
low     575
Name: safety, dtype: int64
class :
 unacc    1209
acc       384
good       69
vgood      65
Name: class, dtype: int64


# Feature engineering 

In [23]:
!pip install category_encoders

     |████████████████████████████████| 86 kB 4.2 MB/s 


In [24]:
import category_encoders as cate

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [25]:
encode=cate.OrdinalEncoder(cols=['buying','maint','doors','persons','lug_boot','safety'])
df=encode.fit_transform(df)

In [26]:
for i in df.columns:
  print(i,':\n',df[i].value_counts())

buying :
 2    432
3    432
4    432
1    431
Name: buying, dtype: int64
maint :
 2    432
3    432
4    432
1    431
Name: maint, dtype: int64
doors :
 2    432
3    432
4    432
1    431
Name: doors, dtype: int64
persons :
 2    576
3    576
1    575
Name: persons, dtype: int64
lug_boot :
 2    576
3    576
1    575
Name: lug_boot, dtype: int64
safety :
 1    576
2    576
3    575
Name: safety, dtype: int64
class :
 unacc    1209
acc       384
good       69
vgood      65
Name: class, dtype: int64


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1727 non-null   int64 
 1   maint     1727 non-null   int64 
 2   doors     1727 non-null   int64 
 3   persons   1727 non-null   int64 
 4   lug_boot  1727 non-null   int64 
 5   safety    1727 non-null   int64 
 6   class     1727 non-null   object
dtypes: int64(6), object(1)
memory usage: 94.6+ KB


In [28]:
encode=cate.OrdinalEncoder(cols=['class'])
df=encode.fit_transform(df)

# Separate dataset

In [29]:
X=df.iloc[:,:-1]
Y=df.iloc[:,-1:]

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test,Y_train,Y_test=train_test_split(X, Y,
                                               test_size=0.2, random_state=42) 

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
rf=RandomForestClassifier(n_estimators=100,random_state=0)

In [34]:
rf.fit(X_train,Y_train)
y_pred=rf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [35]:
from sklearn.metrics import accuracy_score

In [36]:
accuracy_score(Y_test,y_pred)

0.9682080924855492

In [37]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [38]:
print(confusion_matrix(Y_test,y_pred,labels=[1,2,3,4]))
print(classification_report(Y_test,y_pred))

[[237   0   0   0]
 [  0  75   1   1]
 [  0   4  12   1]
 [  0   1   3  11]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       237
           2       0.94      0.97      0.96        77
           3       0.75      0.71      0.73        17
           4       0.85      0.73      0.79        15

    accuracy                           0.97       346
   macro avg       0.88      0.85      0.87       346
weighted avg       0.97      0.97      0.97       346

